In [4]:
!git version

git version 2.34.1


In [5]:
!git config --global user.email “venkatchoudhary.ala@gmail.com”

In [6]:
!git config --global user.name “venkatchoudharyala”

In [5]:
!git clone "https://github.com/venkatchoudharyala/FakeBusterCentralRepo"

fatal: destination path 'FakeBusterCentralRepo' already exists and is not an empty directory.


In [6]:
!ls

FakeBusterCentralRepo  sample_data


In [2]:
%cd FakeBusterCentralRepo

/content/FakeBusterCentralRepo


In [3]:
from tensorflow.keras.models import Model as KerasModel
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU
from tensorflow.keras.optimizers import Adam

IMGWIDTH = 256

class Classifier:
    def _init_():
        self.model = 0

    def predict(self, x):
        if x.size == 0:
            return []
        return self.model.predict(x)

    def fit(self, x, y):
        return self.model.train_on_batch(x, y)

    def get_accuracy(self, x, y):
        return self.model.test_on_batch(x, y)

    def load(self, path):
        self.model.load_weights(path)


class Meso1(Classifier):
    """
    Feature extraction + Classification
    """
    def _init_(self, learning_rate = 0.001, dl_rate = 1):
        self.model = self.init_model(dl_rate)
        optimizer = Adam(lr = learning_rate)
        self.model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['accuracy'])

    def init_model(self, dl_rate):
        x = Input(shape = (IMGWIDTH, IMGWIDTH, 3))

        x1 = Conv2D(16, (3, 3), dilation_rate = dl_rate, strides = 1, padding='same', activation = 'relu')(x)
        x1 = Conv2D(4, (1, 1), padding='same', activation = 'relu')(x1)
        x1 = BatchNormalization()(x1)
        x1 = MaxPooling2D(pool_size=(8, 8), padding='same')(x1)

        y = Flatten()(x1)
        y = Dropout(0.5)(y)
        y = Dense(1, activation = 'sigmoid')(y)
        return KerasModel(inputs = x, outputs = y)


class Meso4(Classifier):
    def _init_(self, learning_rate = 0.001):
        self.model = self.init_model()
        optimizer = Adam(lr = learning_rate)
        self.model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['accuracy'])

    def init_model(self):
        x = Input(shape = (IMGWIDTH, IMGWIDTH, 3))

        x1 = Conv2D(8, (3, 3), padding='same', activation = 'relu')(x)
        x1 = BatchNormalization()(x1)
        x1 = MaxPooling2D(pool_size=(2, 2), padding='same')(x1)

        x2 = Conv2D(8, (5, 5), padding='same', activation = 'relu')(x1)
        x2 = BatchNormalization()(x2)
        x2 = MaxPooling2D(pool_size=(2, 2), padding='same')(x2)

        x3 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x2)
        x3 = BatchNormalization()(x3)
        x3 = MaxPooling2D(pool_size=(2, 2), padding='same')(x3)

        x4 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x3)
        x4 = BatchNormalization()(x4)
        x4 = MaxPooling2D(pool_size=(4, 4), padding='same')(x4)

        y = Flatten()(x4)
        y = Dropout(0.5)(y)
        y = Dense(16)(y)
        y = LeakyReLU(alpha=0.1)(y)
        y = Dropout(0.5)(y)
        y = Dense(1, activation = 'sigmoid')(y)

        return KerasModel(inputs = x, outputs = y)


class MesoInception4(Classifier):
    def _init_(self, learning_rate = 0.001):
        self.model = self.init_model()
        optimizer = Adam(lr = learning_rate)
        self.model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['accuracy'])

    def InceptionLayer(self, a, b, c, d):
        def func(x):
            x1 = Conv2D(a, (1, 1), padding='same', activation='relu')(x)

            x2 = Conv2D(b, (1, 1), padding='same', activation='relu')(x)
            x2 = Conv2D(b, (3, 3), padding='same', activation='relu')(x2)

            x3 = Conv2D(c, (1, 1), padding='same', activation='relu')(x)
            x3 = Conv2D(c, (3, 3), dilation_rate = 2, strides = 1, padding='same', activation='relu')(x3)

            x4 = Conv2D(d, (1, 1), padding='same', activation='relu')(x)
            x4 = Conv2D(d, (3, 3), dilation_rate = 3, strides = 1, padding='same', activation='relu')(x4)

            y = Concatenate(axis = -1)([x1, x2, x3, x4])

            return y
        return func

    def init_model(self):
        x = Input(shape = (IMGWIDTH, IMGWIDTH, 3))

        x1 = self.InceptionLayer(1, 4, 4, 2)(x)
        x1 = BatchNormalization()(x1)
        x1 = MaxPooling2D(pool_size=(2, 2), padding='same')(x1)

        x2 = self.InceptionLayer(2, 4, 4, 2)(x1)
        x2 = BatchNormalization()(x2)
        x2 = MaxPooling2D(pool_size=(2, 2), padding='same')(x2)

        x3 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x2)
        x3 = BatchNormalization()(x3)
        x3 = MaxPooling2D(pool_size=(2, 2), padding='same')(x3)

        x4 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x3)
        x4 = BatchNormalization()(x4)
        x4 = MaxPooling2D(pool_size=(4, 4), padding='same')(x4)

        y = Flatten()(x4)
        y = Dropout(0.5)(y)
        y = Dense(16)(y)
        y = LeakyReLU(alpha=0.1)(y)
        y = Dropout(0.5)(y)
        y = Dense(1, activation = 'sigmoid')(y)

        return KerasModel(inputs = x, outputs = y)

In [4]:
TrainFakePath = "Dataset/Train/Fake"
TrainRealPath = "Dataset/Train/Real"

In [5]:
import os
label = {}
FakeFiles = os.listdir(TrainFakePath)
RealFiles = os.listdir(TrainRealPath)
for file in FakeFiles:
  label[file] = 1
for file in RealFiles:
  label[file] = 0
print(label)

{'fake_5723.jpg': 1, 'fake_47013.jpg': 1, 'fake_10594.jpg': 1, 'fake_7940.jpg': 1, 'fake_62426.jpg': 1, 'fake_16516.jpg': 1, 'fake_49290.jpg': 1, 'fake_46442.jpg': 1, 'fake_54593.jpg': 1, 'fake_226.jpg': 1, 'fake_32691.jpg': 1, 'fake_5157.jpg': 1, 'fake_35689.jpg': 1, 'fake_66861.jpg': 1, 'fake_32410.jpg': 1, 'fake_30709.jpg': 1, 'fake_63793.jpg': 1, 'fake_46258.jpg': 1, 'fake_43581.jpg': 1, 'fake_61116.jpg': 1, 'fake_309.jpg': 1, 'fake_19232.jpg': 1, 'fake_21983.jpg': 1, 'fake_2430.jpg': 1, 'fake_12058.jpg': 1, 'fake_4540.jpg': 1, 'fake_3199.jpg': 1, 'fake_44578.jpg': 1, 'fake_42811.jpg': 1, 'fake_67180.jpg': 1, 'fake_29557.jpg': 1, 'fake_49329.jpg': 1, 'fake_15922.jpg': 1, 'fake_54577.jpg': 1, 'fake_53955.jpg': 1, 'fake_19103.jpg': 1, 'fake_33398.jpg': 1, 'fake_64376.jpg': 1, 'fake_36909.jpg': 1, 'fake_5601.jpg': 1, 'fake_44774.jpg': 1, 'fake_33153.jpg': 1, 'fake_7971.jpg': 1, 'fake_62650.jpg': 1, 'fake_43232.jpg': 1, 'fake_25940.jpg': 1, 'fake_58880.jpg': 1, 'fake_345.jpg': 1, 'fake

In [ ]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.models import Model as KerasModel
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

IMGWIDTH = 256

# Paths to your image directories
directory1 = TrainFakePath
directory2 = TrainRealPath

# List all image files in the directories
image_files_dir1 = [os.path.join(directory1, filename) for filename in os.listdir(directory1) if filename.endswith(".jpg")]
image_files_dir2 = [os.path.join(directory2, filename) for filename in os.listdir(directory2) if filename.endswith(".jpg")]

# Combine image files from both directories
all_image_files = image_files_dir1 + image_files_dir2

# Create labels dictionary where keys are filenames and values are labels
labels_dict = label

# Create arrays to store data and labels
data = []
labels = []

# Inside the loop where you load images
for image_path in all_image_files:
    img = Image.open(image_path)  # Load the image using PIL
    #img = np.array(img) / 255.0  # Normalize pixel values to [0, 1]
    data.append(img)
    labels.append(labels_dict[os.path.basename(image_path)])


# Convert lists to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Split dataset into train and test sets
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define your model
# ...

# Initialize the model
model = MesoInception4(learning_rate=0.001)

# Train the model
model.fit(train_data, train_labels)

# Evaluate the model on test data
accuracy = model.get_accuracy(test_data, test_labels)
print(f"Test accuracy: {accuracy}")
